# Анализ данных онлайн-сервиса по чтению книг

**Цель исследования** – провести анализ данных, содержащих информацию о книгах, с применением SQL-запросов.

**Заказчики анализа** – владельцы сервиса для чтения книг по подписке (стартаперы).

**Конечная цель исследования** – сформулировать ценностное предложение для нового продукта, а также подготовить ответы на следующие вопросы:
- посчитать, сколько книг вышло после 1 января 2000 г. включительно,
- посчитать для каждой книги количество обзоров и среднюю оценку,
- определить издательство, выпуставшиее наибольшее число книг толще 50 страниц (все, что не больше 50 страниц, признается брошюрами), 
- определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками,
- посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

Для проведения анализа в нашем распоряжении имеются следующие пять таблиц:
1) **`books`** – содержит данные о книгах:
- `book_id` – идентификатор книги,
- `author_id` – идентификатор автора,
- `title` – название книги,
- `num_pages` – количество страниц,
- `publication_date` – дата публикации книги,
- `publisher_id` – идентификатор издателя.

2) **`authors`** – содержит данные об авторах:
- `author_id` – идентификатор автора,
- `author` – имя автора.

3) **`publishers`** – содержит данные об издательствах:
- `publisher_id` – идентификатор издательства,
- `publisher` – название издательства.

4) **`ratings`** – содержит данные о пользовательских оценках книг:
- `rating_id` – идентификатор оценки,
- `book_id` – идентификатор книги,
- `username` – имя пользователя, оставившего оценку,
- `rating` – оценка книги.

5) **`reviews`** – содержит данные о пользовательских обзорах на книги:
- `review_id` – идентификатор обзора;
- `book_id` – идентификатор книги;
- `username` – имя пользователя, написавшего обзор;
- `text` – текст обзора.

Схема базы данных сервиса доступна по [ссылке](https://drive.google.com/file/d/1kbkibJAnyWyeHheGAzjqcowvOiJt8gsb/view?usp=share_link).

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подключение-к-базе-данных" data-toc-modified-id="Подключение-к-базе-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подключение к базе данных</a></span></li><li><span><a href="#Изучение-таблиц" data-toc-modified-id="Изучение-таблиц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Изучение таблиц</a></span></li><li><span><a href="#Анализ-данных-(ответы-на-поставленные-вопросы)" data-toc-modified-id="Анализ-данных-(ответы-на-поставленные-вопросы)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ данных (ответы на поставленные вопросы)</a></span></li><li><span><a href="#Рекомендации-для-нового-продукта" data-toc-modified-id="Рекомендации-для-нового-продукта-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Рекомендации для нового продукта</a></span></li></ul></div>

## Подключение к базе данных

Подключимся к базе данных сервиса.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

## Изучение таблиц

Изучим полученные таблицы, выведя первые пять строк и рассчитав количество уникальных значений в каждой из них.

In [2]:
# выведем первые пять строк таблицы "books"
query = '''
        SELECT *
        FROM books
        LIMIT 5
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# посчитаем количество книг в таблице "books"
query = '''
        SELECT COUNT(DISTINCT book_id) AS book_counts
        FROM books
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,book_counts
0,1000


In [4]:
# выведем первые пять строк таблицы "authors"
query = '''
        SELECT *
        FROM authors
        LIMIT 5
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# посчитаем количество авторов в таблице "author"
query = '''
        SELECT COUNT(DISTINCT author_id) AS author_counts
        FROM author
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,author_counts
0,636


In [6]:
# выведем первые пять строк таблицы "publisher"
query = '''
        SELECT *
        FROM publishers
        LIMIT 5
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
# посчитаем количество издательств в таблице "publisher"
query = '''
        SELECT COUNT(DISTINCT publisher_id) AS publisher_counts
        FROM publishers
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,publisher_counts
0,340


In [8]:
# выведем первые пять строк таблицы "ratings"
query = '''
        SELECT *
        FROM ratings
        LIMIT 5
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
# посчитаем количество оценок в таблице "ratings", оставленных пользователями,
# а также количество книг, имеющих такие оценки, и количество пользователей, поставивших оценку
query = '''
        SELECT COUNT(DISTINCT rating_id) AS rating_counts,
               COUNT(DISTINCT book_id) AS books_with_ratings_counts,
               COUNT(DISTINCT username) AS username_counts
        FROM ratings
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,rating_counts,books_with_ratings_counts,username_counts
0,6456,1000,160


In [10]:
# выведем первые пять строк таблицы "reviews"
query = '''
        SELECT *
        FROM reviews
        LIMIT 5
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [11]:
# посчитаем количество обзоров в таблице "reviews", 
# а также количество пользователей, написавших обзоры на книги, 
# и количество книг, на которые были написаны обзоры
query = '''
        SELECT COUNT(DISTINCT review_id) AS review_counts,
               COUNT(DISTINCT username) AS username_counts,
               COUNT(DISTINCT book_id) AS books_with_review_counts
        FROM reviews
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,review_counts,username_counts,books_with_review_counts
0,2793,160,994


**Краткие выводы**

В базе данных сервиса для чтения книг по подписке содержатся 1000 книг, написанных 636 авторами. Все книги в базе данных имеют рейтинги от пользователей и практически на все из них пользователями были написаны обзоры – 994 из 1000. Все оценки и отзывы в сервисе принадлежат 160 пользователям.

Чаще всего пользователи проставляют оценки книгам, чем пишут на них обзоры, – превышение первого показателя над вторым составляет 2.3 раза (или 6.5 тыс. против 2.8 тыс.). Тот, кто проставлял оценки в сервисе, хотя бы один раз написал обзор на ту или иную книгу.

В базе данных представлено 340 издательств, которыми были выпущены те или иные книги, хранящиеся в сервисе.

## Анализ данных (ответы на поставленные вопросы)

Посчитаем, сколько в базе данных хранится книг, изданных после 1 января 2000 г. включительно (`book_counts_after_2020`).

In [12]:
query = '''
        SELECT COUNT(DISTINCT book_id) AS book_counts_after_2020
        FROM books
        WHERE publication_date >= '2000-01-01'
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,book_counts_after_2020
0,821


Посчитаем долю книг, изданных после 1 января 2000 г. включительно, среди всех книг, представленных в сервисе. Для корректности расчета добавим в `SELECT` умножение на `100.0` – чтобы конвертировать тип данных из `int` в `decimal`.

In [13]:
query = '''
        SELECT ROUND((COUNT(DISTINCT book_id)) * 100.0 / (SELECT COUNT(DISTINCT book_id) FROM books)) AS books_percent
        FROM books
        WHERE publication_date > '2000-01-01'
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,books_percent
0,82.0


Посчитаем для каждой книги среднюю оценку (`average_rating`) и количество обзоров (`review_counts`). Значение средней оценки округлим до двух знаков после запятой, а выводимый на экран результат отсортируем по возрастанию – по значениям `average_rating` и `review_counts` соответственно.

In [14]:
query = '''
        SELECT b.book_id, b.title,
               ROUND(AVG(rat.rating), 2) AS average_rating,
               COUNT(DISTINCT rev.review_id) AS review_counts
        FROM books AS b
        LEFT JOIN ratings AS rat on b.book_id = rat.book_id
        LEFT JOIN reviews AS rev on b.book_id = rev.book_id
        GROUP BY b.book_id, 
                 b.title
        ORDER BY 3,
                 4
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,book_id,title,average_rating,review_counts
0,303,Harvesting the Heart,1.50,2
1,316,His Excellency: George Washington,2.00,2
2,371,Junky,2.00,2
3,202,Drowning Ruth,2.00,3
4,915,The World Is Flat: A Brief History of the Twen...,2.25,3
...,...,...,...,...
995,76,Angels Fall,5.00,2
996,913,The Woman in Black,5.00,2
997,444,Moneyball: The Art of Winning an Unfair Game,5.00,3
998,553,School's Out—Forever (Maximum Ride #2),5.00,3


Проверим количество книг в базе данных, не имеющих ни одного обзора, – таковых должно быть 6 шт.

In [15]:
query = '''
        WITH
        table_one AS 
        (SELECT b.book_id,
                ROUND(AVG(rat.rating), 2) AS average_rating,
                COUNT(rev.review_id) AS review_counts
        FROM books AS b
        LEFT JOIN ratings AS rat on b.book_id = rat.book_id
        LEFT JOIN reviews AS rev on b.book_id = rev.book_id
        GROUP BY b.book_id
        ORDER BY 2,
                 3)
                 
        SELECT *
        FROM table_one
        WHERE review_counts = 0
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,book_id,average_rating,review_counts
0,808,3.00,0
1,83,3.67,0
2,221,4.00,0
3,387,4.00,0
4,191,4.00,0
5,672,5.00,0


Определим издательство (`publisher_with_the_most_books`), выпустившее наибольшее число книг толще 50 страниц (книги, содержащие менее 50 страниц, признаны брошюрами).

In [16]:
query = '''
        SELECT publisher AS publisher_with_the_most_books,
               COUNT(b.book_id) AS book_counts
        FROM books AS b
        LEFT JOIN publishers AS p USING(publisher_id)
        WHERE num_pages > 50
        GROUP BY p.publisher
        ORDER BY 2 DESC
        LIMIT 1
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_with_the_most_books,book_counts
0,Penguin Books,42


Определим автора с самой высокой средней оценкой книг (`most_popular_author`) – с учетом книг, которые имеют не менее 50 оценок.

Для этого на первом этапе составим список книг, имеющих 50 и более оценок, – всего таких книг в базе данных 19 шт. Затем соберем таблицу с информацией об авторах этих книг, принадлежащих им рейтингах, рассчитаем средние значения рейтинга, округленные до двух знаков после запятой, а в конечный результат выведем название самого популярного автора и значение его среднего рейтинга.

In [17]:
query = '''
        WITH
        table_one AS
        (SELECT book_id
        FROM (SELECT b.book_id, 
                     COUNT(r.rating) AS rating_counts
              FROM books AS b
              LEFT JOIN ratings AS r USING(book_id)
              GROUP BY b.book_id 
              HAVING COUNT(r.rating) >= 50) AS subtable_one)
                          
        SELECT author AS most_popular_author, 
               ROUND(AVG(rating), 2) AS max_average_rating
        FROM table_one AS tone
        JOIN books AS b USING(book_id)
        JOIN authors AS a USING(author_id)
        JOIN ratings AS r USING(book_id)
        GROUP BY author
        ORDER BY 2 DESC
        LIMIT 1
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,most_popular_author,max_average_rating
0,J.K. Rowling/Mary GrandPré,4.29


Посчитаем среднее количество обзоров от пользователей, оставивших более 50 оценок (`username_average_review`). 

Для этого сначала отберем всех пользователей из таблицы `ratings`, поставивших более 50 оценок, – всего таковых в базе данных 6. После этого посчитаем количество написанных каждым таким пользователем обзоров и на их основе рассчитаем требуемое среднее количество обзоров.

In [18]:
query = '''
        WITH
        table_one AS
        (SELECT username
        FROM (SELECT username,
                     COUNT(rating_id) AS rating
              FROM ratings
              GROUP BY username
              HAVING COUNT(rating_id) > 50) AS subtable_two),
        
        table_two AS 
        (SELECT username,
                COUNT(review_id) AS review_counts
        FROM table_one AS tone
        JOIN reviews AS r USING(username)
        GROUP BY username) 
        
        SELECT ROUND(AVG(review_counts)) AS username_average_review
        FROM table_two
        '''

pd.io.sql.read_sql(sql=text(query), con=con)

,username_average_review
0,24.0


**Краткие выводы**

82% всех книг, имеющихся в сервисе, были выпущены после 1 января 2000 г. включительно. Средняя оценка книг находится в промежутке от 1.5 до 5, а количество написанных обзоров на одну книгу – от 0 до 7 (при этом, как было отмечено ранее, 0.06% книг в сервисе не имеют ни одной рецензии). 

Самым популярным издательством в сервисе является `Penguin Books`, которое выпустило 42 книги. `J.K. Rowling` (совместно с иллюстратором ее книг `Mary GrandPré`) стала самым популярным автором – её книги имеют максимальный рейтинг (4.29).

В среднем самые активные пользователи сервиса, поставившие более 50 оценок, пишут по 24 рецензии.

## Рекомендации для нового продукта

Ограниченность имеющихся данных и проведенных расчетов не позволяют сформировать качественное ценностное предложение для нового продукта. В первую очередь, необходимо получше узнать сервис для чтения по подписке – категории и форматы представленных книг, количество имеющихся подписок, их варианты и цены на них, наличие мобильного приложения и его функциональные возможности. 

Одназначно можно сказать, что в сервисе, скорей всего, пока недостаточно книг для удовлетворения потребностей самых разных пользователей. Поэтому, в первую очередь, менеджменту стартапа рекомендуется увеличить ассортимент и тематику книг, изданных разными издательствами (в том числе переводная литература, если речь идет о российском стартапе). Вторая рекомендация состоит в настройке рекламной кампании (в том числе онлайн-подключения с авторами книг и рубрикой «вопрос-ответ») – поскольку стартапы, как правило, малоизвестны публике. 

Также менеджменту можно рекомендовать создание программы лояльности для подписчиков сервиса. Например, ввести вознаграждение за оставленный отзыв (такая [программа](https://www.chitai-gorod.ru/bookwatch) есть, например, в сети книжных магазинов «Читай-город»). Такие действия могут привлечь новых подписчиков или расширить варианты подписок имеющихся пользователей, поскольку книги с высокими оценками и большим количеством отзывов – благодаря потребительскому поведению и действию социального доказательства – чаще остальных привлекают внимание.